In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
!pip3 install -q scispacy
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

!python3 -m spacy download en

     |████████████████████████████████| 44 kB 3.0 MB/s 
     |████████████████████████████████| 71 kB 5.8 MB/s 
     |████████████████████████████████| 13.5 MB 24.1 MB/s 
     |████████████████████████████████| 6.3 MB 31.4 MB/s 
     |████████████████████████████████| 188 kB 50.2 MB/s 
     |████████████████████████████████| 10.1 MB 55.0 MB/s 
     |████████████████████████████████| 451 kB 55.8 MB/s 
     |████████████████████████████████| 628 kB 47.1 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 33.1 MB 79 kB/s 
     |████████████████████████████████| 125.1 MB 39 kB/s 
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7 MB 516 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔

In [ ]:
!pip3 install -q jsonlines

In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
from pprint import pprint
import os

import spacy
import scispacy
from spacy import displacy
import en_core_sci_sm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import (OrderedDict,Counter, defaultdict)


from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from pprint import pprint
import json
import jsonlines
from ast import literal_eval

/usr/local/lib/python3.7/dist-packages/spacy/util.py:718: UserWarning: [W094] Model 'en_core_sci_sm' (0.3.0) specifies an under-constrained spaCy version requirement: >=2.3.1. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.0.7,<3.1.0
  warnings.warn(warn_msg)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def _get_named_entities(ext_sum):
  nlp = spacy.load("en_ner_bc5cdr_md")
  doc = nlp(ext_sum)
  entities = []
  for ent in doc.ents:
    entities.append(ent.text)
  str_entities = " | ".join(entities)   # a string representation of list of entities with the pipe symbol as a separator
  
  return str_entities

## Read the json lines files containing the pubmed article texts and abstracts, generate named entities from the article_text and the abstract as well after parsing them. 

In [ ]:
def main():
  fileName = "train_data_50k.jsonl"
  DATA_PATH = "pubmed_dataset/pubmed-dataset"

  with open(f"{DATA_PATH}/{fileName}") as f:
    for idx, line in enumerate(f):
      if idx % 1000 == 0:
        print("Iteration: ", idx)

      data = json.loads(line)
      if idx > 2.5e4:
        break
      
      article_text = " ".join(data['article_text'])  ##
      article_abstract = " ".join(data['abstract_text']).replace('<S>', '').replace('</S>', '')   ##

      article_text_named_entities = _get_named_entities(article_text)
      article_abstract_named_entities = _get_named_entities(article_abstract)

      dict_1 = {"article_text" : article_text,
                "article_abstract" : article_abstract,
                "article_text_named_entities" : article_text_named_entities,
                "article_abstract_named_entities" : article_abstract_named_entities
                }
        
      with jsonlines.open("pubmed_w_named_entities_25k.jsonl", "a") as writer:
        writer.write(dict_1)
      writer.close()

  f.close()

In [ ]:
if __name__ == "__main__":
  main()

## Read back a sample object from the jsonlines file saved above

In [ ]:
!pwd

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
fileName = "pubmed_w_named_entities_25k.jsonl"
lst_data = list()

with open(f"{fileName}") as f:
  for idx, line in enumerate(f):
    if idx == 20:
      break
    data = json.loads(line)
    lst_data.append(data)

In [ ]:
lst_data[1]

{'article_abstract': ' backgroundanemia in patients with cancer who are undergoing active therapy is commonly encountered and may worsen quality of life in these patients . the effect of blood transfusion is often temporary and may be associated with serious adverse events .   erythropoiesis - stimulating agents are not effective in 30%50% of patients and may have a negative effect on overall survival.aimsto assess the efficacy and feasibility of intravenous iron therapy in patients with cancer who have non - iron - deficiency anemia and who are undergoing treatment with chemotherapy without the use of erythropoiesis - stimulating agents.methodsadult patients with solid cancers and non - iron - deficiency anemia were included .   ferric sucrose at a dose of 200 mg was given in short intravenous infusions weekly for a total of 12 weeks .   hemoglobin level was measured at baseline , every 3 weeks , and 2 weeks after the last iron infusion ( week 14 ) .   adverse events related to intrav

In [ ]:
lst_data[2]

{'article_abstract': ' tardive dystonia ( td ) is a serious side effect of antipsychotic medications , more with typical antipsychotics , that is potentially irreversible in affected patients .   studies show that newer atypical antipsychotics have a lower risk of td . as a result , many clinicians may have developed a false sense of security when prescribing these medications .   we report a case of 20-year - old male with hyperthymic temperament and borderline intellectual functioning , who developed severe td after low dose short duration exposure to atypical antipsychotic risperidone and then olanzapine .   the goal of this paper is to alert the reader to be judicious and cautious before using casual low dose second generation antipsychotics in patient with no core psychotic features , hyperthymic temperament , or borderline intellectual functioning suggestive of organic brain damage , who are more prone to develop adverse effects such as td and monitor the onset of td in patients 

## https://towardsdatascience.com/understanding-faiss-619bb6db2d1a

In [ ]:
!pip3 install faiss-gpu

     |████████████████████████████████| 90.1 MB 21 kB/s 


In [ ]:
import numpy as np 
import faiss  # this will import the faiss library

In [ ]:
res = faiss.StandardGpuResources()  # use a single GPU

In [ ]:
res

<faiss.swigfaiss.StandardGpuResources; proxy of <Swig Object of type 'faiss::gpu::StandardGpuResources *' at 0x7f0dce6f0e10> >

In [ ]:
dimension = 128    # dimensions of each vector                         
n = 200    # number of vectors                   
np.random.seed(1)             
db_vectors = np.random.random((n, dimension)).astype('float32')

In [ ]:
db_vectors.shape

(200, 128)

In [ ]:
nlist = 5  # number of clusters
quantiser = faiss.IndexFlatL2(dimension)  
index = faiss.IndexIVFFlat(quantiser, dimension, nlist,   faiss.METRIC_L2)

In [ ]:
print(index.is_trained)   # False
index.train(db_vectors)  # train on the database vectors
print(index.ntotal)   # 0
index.add(db_vectors)   # add the vectors and update the index
print(index.is_trained)  # True
print(index.ntotal)   # 200

False
0
True
200


In [ ]:
nprobe = 2  # find 2 most similar clusters
n_query = 10  
k = 3  # return 3 nearest neighbours
np.random.seed(0)   
query_vectors = np.random.random((n_query, dimension)).astype('float32')
distances, indices = index.search(query_vectors, k)

In [ ]:
distances

array([[15.7704525, 16.77301  , 17.171312 ],
       [16.476105 , 18.52229  , 18.811914 ],
       [15.520994 , 16.50026  , 17.069548 ],
       [16.842714 , 17.712343 , 17.828485 ],
       [18.325394 , 18.495459 , 18.684458 ],
       [17.531889 , 18.181791 , 18.331264 ],
       [16.84016  , 17.036638 , 17.091757 ],
       [15.984031 , 16.380919 , 17.270594 ],
       [18.018503 , 18.076103 , 18.766174 ],
       [17.11392  , 17.385286 , 17.657572 ]], dtype=float32)

In [ ]:
indices

array([[185,  35,  96],
       [118,  51, 122],
       [148, 149, 173],
       [175, 177,  50],
       [ 44, 144, 174],
       [156,  74, 151],
       [ 57, 144,  18],
       [ 82,  12,  46],
       [ 52,  73,  59],
       [ 82,  46,  90]])

In [ ]:
# the dict_data passed here is a json object with article_text, article_abstrat, article_text_named_entities and
# article_abstract_named_entities
# K - number of facts per named entity to be summarized (here the facts are retrieved such that they should be semantically similar to the 
# sentences where the named entity appears in)
'''
import itertools

res = faiss.StandardGpuResources()  # use a single GPU


def _get_entity_based_facts(dict_data, top_k=3):
  dict_knowledge_facts_final = defaultdict(list)   # to store facts extracted from the background knowledge bases (UMLS, ICD-11, SNOMED-CT)
  lst_knowledge_facts_total = list()

  lst_named_entities_text = list(set(dict_data['article_text_named_entities'].split(' | ')))
  
  input_text = dict_data['article_text']   # the input_text to be summarized into the abstract---this is what is to be tokenized
  
  input_text_sentences = sent_tokenize(input_text) # this gives us a list of sentences

  lst_query_embeddings = []   # This is for storing avg sentene embeddings of all sentences corresponding to a named entity
  lst_fact_embeddings = []    # to build the faiss index
  for named_entity in lst_named_entities_text:
    sentences_w_named_entity = [sent for sent in input_text_sentences if named_entity in sent]  # Get sentences with the named entity
  
    try:
      facts_complete = PYM.search(named_entity)

      #lst_knowledge_facts = list()
      for fact_str in facts_complete:        
        fact = str(fact_str).split(' # ')[1].strip()   # the actual description linking the named entity pair (disregarding the CUI identifier)
        lst_knowledge_facts_total.append(fact)
    except:
      continue

    sentence_w_named_entities_embeddings = [biobert.sentence_vector(sent).numpy() for sent in sentences_w_named_entity]
    # avg embedding of all sentences where the named entity appears in
    avg_sentence_w_entities_embedding = np.mean(np.array(sentence_w_named_entities_embeddings), axis=0)   # to be used as a query
    
    lst_query_embeddings.append(avg_sentence_w_entities_embedding)   # list of all sentences embeddings for a document (this could be a numpy matrix)
    

  for fact in lst_knowledge_facts_total:
    fact_biobert_embedding = biobert.sentence_vector(fact)
    lst_fact_embeddings.append(fact_biobert_embedding)

  np_query_embeddings = np.asarray(lst_query_embeddings)
  np_fact_embeddings = np.asarray(lst_fact_embeddings)

  d = np_query_embeddings.shape[1]   # grab the vector dimension
  #print(d)
  #raise KeyboardInterrupt

  quantiser = faiss.IndexFlatL2(d)  
  index = faiss.IndexIVFFlat(quantiser, d, faiss.METRIC_L2)
  # make it an IVF GPU index
  index = faiss.index_cpu_to_gpu(res, 0, index)  # added on 1129

  index.train(np_fact_embeddings)  # train on the database vectors
  #print(index.ntotal)   # 0
  index.add(np_fact_embeddings)   # add the vectors and update the index
  #######################################################
  '''
  index = faiss.IndexFlatL2(d)   # build the index
  #print(index.is_trained)
  index.add(np_fact_embeddings)                  # add knowledge (fact) vectors to the index
  #print(index.ntotal)
  '''

  #k = 4                          # we want to see 4 nearest neighbors
  distances, indices = index.search(np_query_embeddings, top_k)     # actual search, I is the index
  #print(indices[:5])                   # neighbors of the 5 first queries
  #print(indices[-5:])                  # neighbors of the 5 last queries

  total_indices = list(itertools.chain(*indices))
  final_facts = list(set([lst_knowledge_facts_total[idx] for idx in total_indices]))
  knowledge_facts_total_str = " | ".join(final_facts)  # Join all the retrieved facts with the pipe separator

  return knowledge_facts_total_str

'''


## Now, onto the actual knowledge retriever (from UMLS etc) component

In [ ]:
#!pip3 install -q Owlready2k
!pip3 install owlready2

     |████████████████████████████████| 23.8 MB 1.3 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.35-cp37-cp37m-linux_x86_64.whl size=20432789 sha256=0edc01938b7996ea13fd612b5b61d778b3ec30d7db3f323f79d6f1f6d9903d4c
  Stored in directory: /root/.cache/pip/wheels/a6/85/8f/4f254dc1d3b7901c23004544f7214748607d8b0c2c02b3c868
Successfully built owlready2


In [ ]:
!pip3 install -q transformers

     |████████████████████████████████| 3.3 MB 6.8 MB/s 
     |████████████████████████████████| 895 kB 57.4 MB/s 
     |████████████████████████████████| 3.3 MB 63.3 MB/s 
     |████████████████████████████████| 61 kB 568 kB/s 
     |████████████████████████████████| 596 kB 58.9 MB/s 


In [ ]:
import transformers

In [ ]:
!pip3 install -q biobert-embedding==0.1.2

     |████████████████████████████████| 748.9 MB 661 bytes/s 
     |████████████████████████████████| 123 kB 65.3 MB/s 
     |████████████████████████████████| 131 kB 62.3 MB/s 
     |████████████████████████████████| 8.4 MB 53.8 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 138 kB 75.9 MB/s 
     |████████████████████████████████| 127 kB 68.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.2.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
from biobert_embedding.embedding import BiobertEmbedding

In [ ]:
biobert = BiobertEmbedding()

In [ ]:
from owlready2 import *
from owlready2.pymedtermino2 import *
from owlready2.pymedtermino2.umls import *

In [ ]:
#default_world.set_backend(filename = "pym.sqlite3")
import_umls("umls-2021AA-full.zip", terminologies = ["ICD10", "SNOMEDCT_US", "CUI"])

Importing UMLS from umls-2021AA-full.zip with Python version 3.7.12 and Owlready version 2-0.35...
Full UMLS release - importing UMLS from inner Zip file 2021AA-full/2021aa-1-meta.nlm...
  Parsing 2021AA/META/MRSTY.RRF.gz as MRSTY with encoding UTF-8
  Parsing 2021AA/META/MRRANK.RRF.gz as MRRANK with encoding UTF-8
  Parsing 2021AA/META/MRCONSO.RRF.aa.gz as MRCONSO with encoding UTF-8
  Parsing 2021AA/META/MRCONSO.RRF.ab.gz as MRCONSO with encoding UTF-8
  Parsing 2021AA/META/MRDEF.RRF.gz as MRDEF with encoding UTF-8
Full UMLS release - importing UMLS from inner Zip file 2021AA-full/2021aa-2-meta.nlm...
  Parsing 2021AA/META/MRREL.RRF.aa.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ab.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ac.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRREL.RRF.ad.gz as MRREL with encoding UTF-8
  Parsing 2021AA/META/MRSAT.RRF.aa.gz as MRSAT with encoding UTF-8
  Parsing 2021AA/META/MRSAT.RRF.ab.gz as MRSAT with en

get_ontology("http://PYM/")

In [ ]:
PYM = get_ontology("http://PYM/").load()
default_world.save()
CUI = PYM["CUI"]

### Method to retrieve top-K facts from background KB for each sentence having the named entities extracted. First, tokenize the input_text into sentences and for each named entity, grab the top-K facts and measure semantic similarity using FAISS with the sentence in which the named entity appears in. Ablation study to be conducted with different values of K (5, 10, 15)

In [ ]:
from sklearn import metrics
from scipy.spatial import distance

In [ ]:
# the dict_data passed here is a json object with article_text, article_abstrat, article_text_named_entities and
# article_abstract_named_entities
# K - number of facts per named entity to be summarized (here the facts are retrieved such that they should be semantically similar to the 
# sentences where the named entity appears in)


def _get_entity_based_facts(dict_data, top_k=5):
  dict_knowledge_facts_final = defaultdict(list)   # to store facts extracted from the background knowledge bases (UMLS, ICD-11, SNOMED-CT)
  lst_knowledge_facts_total = list()

  lst_named_entities_text = list(set(dict_data['article_text_named_entities'].split(' | ')))
  
  input_text = dict_data['article_text']   # the input_text to be summarized into the abstract---this is what is to be tokenized
  
  input_text_sentences = sent_tokenize(input_text) # this gives us a list of sentences
  for named_entity in lst_named_entities_text:
    sentences_w_named_entity = [sent for sent in input_text_sentences if named_entity in sent]  # Get sentences with the named entity
  
    try:
      facts_complete = PYM.search(named_entity)

      lst_knowledge_facts = list()
      for fact_str in facts_complete:        
        fact = str(fact_str).split(' # ')[1].strip()   # the actual description linking the named entity pair (disregarding the CUI identifier)
        lst_knowledge_facts.append(fact)
    except:
      continue

    sentence_w_named_entities_embeddings = [biobert.sentence_vector(sent).numpy() for sent in sentences_w_named_entity]
    # avg embedding of all sentences where the named entity appears in
    avg_sentence_w_entities_embedding = np.mean(np.array(sentence_w_named_entities_embeddings), axis=0)   # to be used as a query

    # grab top-k facts with high cosine similarity with the avg_sentence_embedding of all sentencew where the named entity appears in
    dict_fact_cosine = {}
    for fact in lst_knowledge_facts:
      fact_biobert_embedding = biobert.sentence_vector(fact)
      cosine_sim = 1 - distance.cosine(fact_biobert_embedding, avg_sentence_w_entities_embedding)   # cosine distance between fact embedding and avg sent
      dict_fact_cosine[fact] = cosine_sim

    # sort the facts based on cosine_sim and return the top-k facts
    top_k_facts = list({k: v for k, v in sorted(dict_fact_cosine.items(), 
                                                key=lambda item: item[1], reverse=True)}.keys())[:top_k]


    lst_knowledge_facts_total += top_k_facts   # This is for the entire input_text

  knowledge_facts_total_str = " | ".join(lst_knowledge_facts_total)  # Join all the facts with the pipe separator


  return knowledge_facts_total_str




In [ ]:
def main():
  input_fileName = "pubmed_w_named_entities_25k.jsonl"
  output_fileName = "pubmed_w_named_entities_knowledge_facts_25k.jsonl"

  with open(f"{input_fileName}") as f:
    for idx, line in enumerate(f):
      if idx > 1524:
        try: 
          if idx % 100 == 0:
            print("Iteration: ", idx)
          dict_data = json.loads(line)
          article_text_facts = _get_entity_based_facts(dict_data)   # call to the knowledge (fact) retriever method


          dict_1 = {"article_text" : dict_data['article_text'],
                    "article_abstract" : dict_data['article_abstract'],
                    "article_text_named_entities" : dict_data['article_text_named_entities'],
                    "article_abstract_named_entities" : dict_data['article_abstract_named_entities'],
                    "article_text_facts" : article_text_facts
                    }
              
          with jsonlines.open(output_fileName, "a") as writer:
            writer.write(dict_1)
          writer.close()
        except:
          continue

  f.close()

In [ ]:
if __name__ == "__main__":
  main()

Iteration:  1600
Iteration:  1700
Iteration:  1800
Iteration:  1900
Iteration:  2000
Iteration:  2100
Iteration:  2200
Iteration:  2300
Iteration:  2400
Iteration:  2500
Iteration:  2600
Iteration:  2700
Iteration:  2800
Iteration:  2900
Iteration:  3000
Iteration:  3100
Iteration:  3200
Iteration:  3300
Iteration:  3400
Iteration:  3500
Iteration:  3600
Iteration:  3700
Iteration:  3800
Iteration:  3900
Iteration:  4000
Iteration:  4100
Iteration:  4200
Iteration:  4300
Iteration:  4400
Iteration:  4500
Iteration:  4600
Iteration:  4700
Iteration:  4800
